### Reformat the processed netcdf files so that lat/lon variables are 1D not 2D and have coordinate variables with lat/lon dimensions

In [4]:
import os
import netCDF4
import numpy as np

src_dir = '/Users/nberg/projects/doe/data/processed/'
inv_dir = '/Users/nberg/projects/doe/data/invariant/'
datasets = ['wrf', 'livneh', 'loca_wrf', 'loca_livneh']
variables = ['tmax_april', 'annual_pr']
periods = ['hist', 'fut']

for dataset in datasets:
    for variable in variables:
        for period in periods:
            src_fl = src_dir+'{0}_{1}_1991-2000_{2}.nc'.format(dataset, period, variable)
            if os.path.exists(src_fl):
            
                # Read in this dataset's gridded data 
                ncfile = netCDF4.Dataset(src_fl, 'r')
                lat2d = ncfile.variables['lat'][:,:]
                lon2d = ncfile.variables['lon'][:,:]
                if variable == 'tmax_april':
                    tmax = ncfile.variables['tmax'][:,:]
                    # Convert from K to C
                    tmax = tmax - 273.15 
                if variable == 'annual_pr':
                    pr = ncfile.variables['pr'][:,:]
                ncfile.close()
                
                # Read in the CA land mask for this dataset and carve out CA
                mask_ncfile = netCDF4.Dataset(inv_dir+'{0}_CA_mask.nc'.format('-'.join(dataset.split('_')), 'r'))
                landmask = mask_ncfile.variables['landmask'][:,:]
                landmask_bool = np.array(landmask, dtype=bool)
                mask_ncfile.close()
                
                if variable == 'tmax_april':
                    tmax_ca = np.ma.masked_where(landmask == 0, tmax)
                if variable == 'annual_pr':
                    pr_ca = np.ma.masked_where(landmask == 0, pr)
                
                # Create 1D arrays of lat and lon
                lat1d = lat2d[:,0]
                lon1d = lon2d[0,:]
                nlat, nlon = len(lat1d), len(lon1d)
                
                # Output reformatted netcdf files with metadata!
                dest_fl = src_dir+'{0}_{1}_1991-2000_{2}_CA_mask.nc'.format(dataset, period, variable)
                if os.path.exists(dest_fl):
                    os.remove(dest_fl)
                    
                ncfile_out = netCDF4.Dataset(dest_fl, 'w')
                ncfile_out.createDimension('lat', nlat)
                ncfile_out.createDimension('lon', nlon)
                lat_out = ncfile_out.createVariable('lat', 'f4', ('lat'),)
                lon_out = ncfile_out.createVariable('lon', 'f4', ('lon'),)
                if variable == 'tmax_april':
                    tmax_out = ncfile_out.createVariable('tmax', 'f4', ('lat', 'lon'),)
                    setattr(tmax_out, 'units', 'degC')
                if variable == 'annual_pr':
                    pr_out = ncfile_out.createVariable('pr', 'f4', ('lat', 'lon'),)
                    setattr(pr_out, 'units', 'mm/day')
                
                setattr(lat_out, 'units', 'degrees_north')
                setattr(lon_out, 'units', 'degrees_east')
                
                lat_out[:] = lat1d[:]
                lon_out[:] = lon1d[:]
                if variable == 'tmax_april':
                    tmax_out[:,:] = tmax_ca[:,:]
                if variable == 'annual_pr':
                    pr_out[:,:] = pr_ca[:,:]
                ncfile_out.close()
                print(dest_fl)
                

/Users/nberg/projects/doe/data/processed/wrf_hist_1991-2000_tmax_april_CA_mask.nc
/Users/nberg/projects/doe/data/processed/wrf_fut_1991-2000_tmax_april_CA_mask.nc
/Users/nberg/projects/doe/data/processed/wrf_hist_1991-2000_annual_pr_CA_mask.nc
/Users/nberg/projects/doe/data/processed/wrf_fut_1991-2000_annual_pr_CA_mask.nc
/Users/nberg/projects/doe/data/processed/livneh_hist_1991-2000_tmax_april_CA_mask.nc
/Users/nberg/projects/doe/data/processed/livneh_hist_1991-2000_annual_pr_CA_mask.nc
/Users/nberg/projects/doe/data/processed/loca_wrf_hist_1991-2000_tmax_april_CA_mask.nc
/Users/nberg/projects/doe/data/processed/loca_wrf_fut_1991-2000_tmax_april_CA_mask.nc
/Users/nberg/projects/doe/data/processed/loca_wrf_hist_1991-2000_annual_pr_CA_mask.nc
/Users/nberg/projects/doe/data/processed/loca_wrf_fut_1991-2000_annual_pr_CA_mask.nc
/Users/nberg/projects/doe/data/processed/loca_livneh_hist_1991-2000_tmax_april_CA_mask.nc
/Users/nberg/projects/doe/data/processed/loca_livneh_fut_1991-2000_tmax_a

/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in greater
/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in greater
